In [2]:
# NBR 계산 셀
import rasterio
import numpy as np
import glob
import os
from tqdm import tqdm

# 경로 설정
input_dir = r"D:\Landslide\data\raw\sentinel-2\gyeongnam\post_unzip"
output_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 마스킹할 클래스들 (구름, 해역, 그림자 등)
invalid_classes = [0, 1, 3, 6, 8, 9, 10]

def load_band(granule_path, band_name, resolution='20m'):
    """특정 밴드 로드 (R20m 폴더 우선)"""
    img_data_path = os.path.join(granule_path, "IMG_DATA")
    
    # R20m 폴더에서 찾기
    res_folder = os.path.join(img_data_path, f"R{resolution}")
    
    if os.path.exists(res_folder):
        pattern = os.path.join(res_folder, f"*_{band_name}_{resolution}.jp2")
        files = glob.glob(pattern)
        if files:
            return files[0]
    
    # 해상도 표시 없는 파일명 시도
    pattern = os.path.join(img_data_path, "**", f"*_{band_name}.jp2")
    files = glob.glob(pattern, recursive=True)
    if files:
        return files[0]
    
    return None

def calculate_nbr(safe_folder):
    """단일 SAFE 폴더에 대해 NBR 계산
    
    NBR (Normalized Burn Ratio) = (NIR - SWIR2) / (NIR + SWIR2)
    - NIR: B8A (Sentinel-2, 865nm)
    - SWIR2: B12 (Sentinel-2, 2190nm)
    """
    folder_name = os.path.basename(safe_folder)
    print(f"\n처리 중: {folder_name}")
    
    # GRANULE 폴더 찾기
    granule_pattern = os.path.join(safe_folder, "GRANULE", "*")
    granule_dirs = glob.glob(granule_pattern)
    
    if not granule_dirs:
        print(f"  ✗ GRANULE 폴더 없음")
        return False
    
    granule_path = granule_dirs[0]
    
    # 필요한 밴드 파일 경로 찾기
    bands_needed = {
        'B8A': 'B8A',  # NIR (Near Infrared)
        'B12': 'B12',  # SWIR2 (Shortwave Infrared 2)
        'SCL': 'SCL'   # Scene Classification Layer
    }
    
    band_files = {}
    for key, band_name in bands_needed.items():
        file_path = load_band(granule_path, band_name, '20m')
        if file_path:
            band_files[key] = file_path
        else:
            print(f"  ✗ {band_name}: 파일 없음")
            return False
    
    if len(band_files) != len(bands_needed):
        print(f"  ✗ 필요한 밴드가 모두 없음")
        return False
    
    # 밴드 데이터 읽기
    with rasterio.open(band_files['B8A']) as src:
        b8a = src.read(1).astype(np.float32)
        crs = src.crs
        transform = src.transform
        width = src.width
        height = src.height
    
    with rasterio.open(band_files['B12']) as src:
        b12 = src.read(1).astype(np.float32)
    
    with rasterio.open(band_files['SCL']) as src:
        scl = src.read(1)
    
    # 정규화 (DN -> 반사도) - offset 없이 처리
    quantification_value = 10000
    b8a = b8a / quantification_value
    b12 = b12 / quantification_value
    
    # 음수 값을 0으로 클리핑 (물리적으로 불가능한 반사도 제거)
    b8a = np.clip(b8a, 0, None)
    b12 = np.clip(b12, 0, None)
    
    # 0으로 나누기 방지를 위한 epsilon
    epsilon = 1e-6
    
    # === 마스킹 로직 ===
    # 1. SCL 기반 마스크 (구름, 그림자, 물 등)
    scl_mask = np.isin(scl, invalid_classes)
    
    # 2. 분모가 0에 가까운 경우 마스크
    b8a_b12_sum = b8a + b12
    denominator_invalid_mask = b8a_b12_sum < epsilon
    
    # NBR 계산 (마스킹 전)
    # 분모를 epsilon으로 대체하여 계산
    b8a_b12_sum_safe = np.where(b8a_b12_sum < epsilon, epsilon, b8a_b12_sum)
    nbr = (b8a - b12) / b8a_b12_sum_safe
    
    # 3. inf 또는 nan 값 마스크
    invalid_values_mask = ~np.isfinite(nbr)
    
    # 4. 최종 마스크 통합
    final_mask = scl_mask | denominator_invalid_mask | invalid_values_mask
    
    # 최종 마스킹 적용
    nbr[final_mask] = np.nan
    
    # 마스킹 통계
    total_pixels = final_mask.size
    scl_masked = np.sum(scl_mask)
    denominator_masked = np.sum(denominator_invalid_mask)
    invalid_masked = np.sum(invalid_values_mask)
    total_masked = np.sum(final_mask)
    masked_ratio = (total_masked / total_pixels) * 100
    
    # GeoTIFF로 저장
    output_filename = f"{folder_name}_NBR.tif"
    output_path = os.path.join(output_dir, output_filename)
    
    # GeoTIFF 프로파일 설정
    profile = {
        'driver': 'GTiff',
        'dtype': rasterio.float32,
        'count': 1,
        'width': width,
        'height': height,
        'crs': crs,
        'transform': transform,
        'compress': 'lzw',
        'nodata': np.nan
    }
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(nbr, 1)
        dst.set_band_description(1, 'NBR')
        dst.update_tags(1, 
                       BAND_NAME='NBR',
                       DESCRIPTION='Normalized Burn Ratio for Sentinel-2',
                       FORMULA='(B8A - B12) / (B8A + B12)',
                       TOTAL_MASKED_RATIO=f'{masked_ratio:.1f}%',
                       SCL_MASKED_PIXELS=str(scl_masked),
                       DENOMINATOR_INVALID_PIXELS=str(denominator_masked),
                       INVALID_VALUES_PIXELS=str(invalid_masked))
    
    print(f"  ✅ {output_filename}")
    print(f"     NBR: {np.nanmin(nbr):.3f} ~ {np.nanmax(nbr):.3f}")
    print(f"     마스킹: {masked_ratio:.1f}% (SCL:{scl_masked}, Denom:{denominator_masked}, Invalid:{invalid_masked})")
    
    return True

# 메인 실행
print(f"=== NBR 계산 시작 ===")
print(f"입력 경로: {input_dir}")
print(f"출력 경로: {output_dir}\n")

# 모든 SAFE 폴더 찾기
safe_folders = [d for d in glob.glob(os.path.join(input_dir, "*")) if os.path.isdir(d)]
print(f"발견된 SAFE 폴더: {len(safe_folders)}개\n")

success_count = 0
fail_count = 0
failed_list = []

for idx, safe_folder in enumerate(safe_folders, 1):
    try:
        print(f"[{idx}/{len(safe_folders)}]", end=" ")
        if calculate_nbr(safe_folder):
            success_count += 1
        else:
            fail_count += 1
            failed_list.append(os.path.basename(safe_folder))
    except Exception as e:
        print(f"  ✗ 오류: {str(e)[:100]}")
        fail_count += 1
        failed_list.append(os.path.basename(safe_folder))

print(f"\n{'='*60}")
print(f"=== NBR 계산 완료 ===")
print(f"성공: {success_count}개")
print(f"실패: {fail_count}개")
print(f"출력 경로: {output_dir}")

if failed_list:
    print(f"\n실패 목록 ({len(failed_list)}개):")
    for i, name in enumerate(failed_list[:10], 1):
        print(f"  {i}. {name}")
    if len(failed_list) > 10:
        print(f"  ... 외 {len(failed_list) - 10}개")

=== NBR 계산 시작 ===
입력 경로: D:\Landslide\data\raw\sentinel-2\gyeongnam\post_unzip
출력 경로: D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR

발견된 SAFE 폴더: 150개

[1/150] 
처리 중: 20151028_R103_T52SCD
  ✅ 20151028_R103_T52SCD_NBR.tif
     NBR: -0.842 ~ 0.625
     마스킹: 78.6% (SCL:23701436, Denom:13361076, Invalid:0)
[2/150] 
처리 중: 20151028_R103_T52SDE
  ✅ 20151028_R103_T52SDE_NBR.tif
     NBR: -0.846 ~ 0.663
     마스킹: 3.4% (SCL:1026123, Denom:0, Invalid:0)
[3/150] 
처리 중: 20160309_R003_T52SCH
  ✅ 20160309_R003_T52SCH_NBR.tif
     NBR: -0.773 ~ 0.488
     마스킹: 69.6% (SCL:20986854, Denom:18615084, Invalid:0)
[4/150] 
처리 중: 20160408_R003_T52SCE
  ✅ 20160408_R003_T52SCE_NBR.tif
     NBR: -0.691 ~ 0.682
     마스킹: 1.3% (SCL:406717, Denom:0, Invalid:0)
[5/150] 
처리 중: 20160408_R003_T52SDE
  ✅ 20160408_R003_T52SDE_NBR.tif
     NBR: -0.670 ~ 0.610
     마스킹: 55.1% (SCL:16597739, Denom:16377370, Invalid:0)
[6/150] 
처리 중: 20160415_R103_T52SCD
  ✅ 20160415_R103_T52SCD_NBR.tif
     NBR: -0.799 ~ 0.615
 

In [4]:
# ============================================================
# NBR 래스터 산지지역 마스킹
# ============================================================
import rasterio
import rasterio.mask
import geopandas as gpd
import numpy as np
import pandas as pd
import glob
import os
from tqdm import tqdm

# ============================================================
# 1. 기본 설정 및 경로 정의
# ============================================================
# 원본 NBR 래스터 파일 경로
input_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR"

# 마스킹된 결과를 저장할 경로
output_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR_masked"

# 산지지역 마스크 파일 경로
mask_path = r"D:\Landslide\data\processed\gyeongnam\terrain_information\gyeongnam_forest_mask.gpkg"

# 출력 디렉토리 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("=" * 70)
print("=== NBR 산지지역 마스킹 시작 ===")
print("=" * 70)
print(f"입력 경로: {input_dir}")
print(f"출력 경로: {output_dir}")
print(f"마스크 파일: {mask_path}\n")

# ============================================================
# 2. 산지지역 마스크 로드
# ============================================================
print("[1/3] 산지지역 마스크 로드 중...")

if not os.path.exists(mask_path):
    raise FileNotFoundError(f"마스크 파일을 찾을 수 없습니다: {mask_path}")

# GeoPackage 읽기
gdf_mask = gpd.read_file(mask_path)

print(f"   ✅ 마스크 로드 완료")
print(f"      피처 수: {len(gdf_mask)}개")
print(f"      CRS: {gdf_mask.crs}")
print(f"      총 면적: {gdf_mask.geometry.area.sum() / 1e6:.2f} km²\n")

# ============================================================
# 3. 모든 NBR 래스터 파일 마스킹
# ============================================================
print("[2/3] NBR 래스터 파일 마스킹 중...")

# 모든 TIF 파일 목록 가져오기
all_tif_files = sorted(glob.glob(os.path.join(input_dir, "*.tif")))
print(f"   발견된 TIF 파일: {len(all_tif_files)}개\n")

# 마스킹 결과 저장
masking_results = []
success_count = 0
failed_count = 0
failed_files = []

for idx, tif_path in enumerate(tqdm(all_tif_files, desc="   처리 중"), 1):
    filename = os.path.basename(tif_path)
    
    try:
        # 래스터 파일 열기
        with rasterio.open(tif_path) as src:
            src_crs = src.crs
            
            # 마스크를 래스터와 동일한 CRS로 변환 (필요시)
            if gdf_mask.crs != src_crs:
                gdf_mask_reprojected = gdf_mask.to_crs(src_crs)
            else:
                gdf_mask_reprojected = gdf_mask
            
            # 마스크 적용하여 래스터 자르기
            # crop=True: 마스크 영역에 맞게 래스터 크기 조정
            # all_touched=False: 폴리곤 내부에 중심이 있는 픽셀만 포함 (더 보수적)
            out_image, out_transform = rasterio.mask.mask(
                src, 
                gdf_mask_reprojected.geometry, 
                crop=True,
                all_touched=False,
                nodata=np.nan
            )
            
            # 메타데이터 업데이트
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
                "nodata": np.nan,
                "compress": "lzw"
            })
        
        # 마스킹 통계 계산
        total_pixels = out_image.size
        valid_pixels = np.sum(~np.isnan(out_image))
        masked_pixels = total_pixels - valid_pixels
        valid_ratio = (valid_pixels / total_pixels) * 100
        
        # 결과 저장
        output_filename = f"masked_{filename}"
        output_path = os.path.join(output_dir, output_filename)
        
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)
            dest.set_band_description(1, 'NBR_Forest_Masked')
            dest.update_tags(1,
                           MASKING='Forest Area Mask Applied',
                           MASK_FILE=os.path.basename(mask_path),
                           VALID_PIXELS=str(valid_pixels),
                           MASKED_PIXELS=str(masked_pixels),
                           VALID_RATIO=f'{valid_ratio:.2f}%')
        
        # 결과 기록
        masking_results.append({
            'filename': filename,
            'total_pixels': total_pixels,
            'valid_pixels': valid_pixels,
            'masked_pixels': masked_pixels,
            'valid_ratio': valid_ratio,
            'output_file': output_filename
        })
        
        success_count += 1
        
    except Exception as e:
        error_msg = str(e)[:100]
        print(f"\n   ✗ [{idx}] {filename}: 오류 - {error_msg}")
        failed_count += 1
        failed_files.append((filename, error_msg))

print(f"\n   ✅ 마스킹 완료: {success_count}개 성공, {failed_count}개 실패\n")

# ============================================================
# 4. 결과 요약 및 저장
# ============================================================
print("[3/3] 결과 요약 및 저장 중...")

# 마스킹 결과 DataFrame
df_masking = pd.DataFrame(masking_results)

if len(df_masking) > 0:
    # 통계 요약
    print(f"\n   [마스킹 통계]")
    print(f"   - 평균 유효 픽셀 비율: {df_masking['valid_ratio'].mean():.2f}%")
    print(f"   - 총 유효 픽셀: {df_masking['valid_pixels'].sum():,}개")
    print(f"   - 총 마스킹된 픽셀: {df_masking['masked_pixels'].sum():,}개")
    
    # CSV 저장
    masking_csv = os.path.join(output_dir, "masking_results.csv")
    df_masking.to_csv(masking_csv, index=False, encoding='utf-8-sig')
    print(f"\n   ✅ 마스킹 결과 저장: {masking_csv}")

# 실패 목록 저장
if failed_files:
    df_failed = pd.DataFrame(failed_files, columns=['filename', 'error'])
    failed_csv = os.path.join(output_dir, "failed_files.csv")
    df_failed.to_csv(failed_csv, index=False, encoding='utf-8-sig')
    print(f"   ✅ 실패 목록 저장: {failed_csv}")

# 최종 요약 리포트
report_path = os.path.join(output_dir, "masking_report.txt")
with open(report_path, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("NBR 산지지역 마스킹 리포트\n")
    f.write("="*70 + "\n\n")
    f.write(f"작업 일시: {pd.Timestamp.now()}\n")
    f.write(f"입력 경로: {input_dir}\n")
    f.write(f"출력 경로: {output_dir}\n")
    f.write(f"마스크 파일: {mask_path}\n\n")
    
    f.write(f"[마스크 정보]\n")
    f.write(f"  피처 수: {len(gdf_mask)}개\n")
    f.write(f"  CRS: {gdf_mask.crs}\n")
    f.write(f"  총 면적: {gdf_mask.geometry.area.sum() / 1e6:.2f} km²\n\n")
    
    f.write(f"[처리 결과]\n")
    f.write(f"  총 파일 수: {len(all_tif_files)}개\n")
    f.write(f"  마스킹 완료: {success_count}개\n")
    f.write(f"  실패: {failed_count}개\n\n")
    
    if len(df_masking) > 0:
        f.write(f"[마스킹 통계]\n")
        f.write(f"  평균 유효 픽셀 비율: {df_masking['valid_ratio'].mean():.2f}%\n")
        f.write(f"  총 유효 픽셀: {df_masking['valid_pixels'].sum():,}개\n")
        f.write(f"  총 마스킹된 픽셀: {df_masking['masked_pixels'].sum():,}개\n\n")
    
    if failed_files:
        f.write(f"[실패한 파일]\n")
        for filename, error in failed_files[:20]:
            f.write(f"  - {filename}: {error}\n")
        if len(failed_files) > 20:
            f.write(f"  ... 외 {len(failed_files)-20}개\n")

print(f"   ✅ 리포트 저장: {report_path}")

print("\n" + "="*70)
print("=== 마스킹 완료 ===")
print("="*70)
print(f"\n 총 {success_count}개 파일이 마스킹되어 저장되었습니다.")
print(f" 출력 경로: {output_dir}")

=== NBR 산지지역 마스킹 시작 ===
입력 경로: D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR
출력 경로: D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR_masked
마스크 파일: D:\Landslide\data\processed\gyeongnam\terrain_information\gyeongnam_forest_mask.gpkg

[1/3] 산지지역 마스크 로드 중...
   ✅ 마스크 로드 완료
      피처 수: 1개
      CRS: EPSG:5179
      총 면적: 6897.18 km²

[2/3] NBR 래스터 파일 마스킹 중...
   발견된 TIF 파일: 150개



   처리 중:   2%|▏         | 3/150 [01:12<56:57, 23.25s/it]  


   ✗ [3] 20160309_R003_T52SCH_NBR.tif: 오류 - Input shapes do not overlap raster.


   처리 중: 100%|██████████| 150/150 [1:39:26<00:00, 39.78s/it]


   ✅ 마스킹 완료: 149개 성공, 1개 실패

[3/3] 결과 요약 및 저장 중...

   [마스킹 통계]
   - 평균 유효 픽셀 비율: 32.19%
   - 총 유효 픽셀: 793,906,384개
   - 총 마스킹된 픽셀: 1,791,710,877개

   ✅ 마스킹 결과 저장: D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR_masked\masking_results.csv
   ✅ 실패 목록 저장: D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR_masked\failed_files.csv
   ✅ 리포트 저장: D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR_masked\masking_report.txt

=== 마스킹 완료 ===

 총 149개 파일이 마스킹되어 저장되었습니다.
 출력 경로: D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR_masked


In [ ]:
# dNBR 계산 셀
import rasterio
import numpy as np
import pandas as pd
import glob
import os
import re
from rasterio.warp import reproject, Resampling
from tqdm import tqdm

reference_csv = r"D:\Landslide\data\raw\산불발생이력\s2_matches-15.csv"
pre_id_column = 'pre_product_id'
post_id_column = 'post_product_id'

pre_masked_NBR_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\pre\NBR"
post_masked_NBR_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR"

output_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\dNBR_unmaksed"
unpaired_log = os.path.join(output_dir, "unpaired.log")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("="*70)
print("=== dNBR 계산 시작 ===")
print("="*70)
print(f"CSV 파일: {reference_csv}")
print(f"Pre-NBR 경로: {pre_masked_NBR_dir}")
print(f"Post-NBR 경로: {post_masked_NBR_dir}")
print(f"출력 경로: {output_dir}\n")

# ============================================================
# 1단계: CSV 파일 로드 및 Pair 리스트 생성
# ============================================================
print("[1/4] CSV 파일 로드 및 Pair 리스트 생성 중...")

df_ref = pd.read_csv(reference_csv, encoding='utf-8-sig')
print(f"   총 레코드 수: {len(df_ref)}개")

# 짧은 파일명 추출 함수 (20151028_R103_T52SCD 형식으로 변환)
def extract_short_name(product_id):
    """
    S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010
    -> 20151028_R103_T52SCD
    """
    if pd.isna(product_id) or product_id == '':
        return None

    match = re.search(r'(\d{8})T\d+_N\d+_(R\d+)_(T\w+)_', str(product_id))
    if match:
        date = match.group(1)
        orbit = match.group(2)
        tile = match.group(3)
        return f"{date}_{orbit}_{tile}"
    return None

# 파일명 디렉토리 매핑 생성
def build_file_mapping(directory):
    """디렉토리 내 파일명 -> 전체 경로 매핑
    
    다음 패턴들을 모두 지원:
    1. masked_20151028_R103_T52SCD_NBR.tif (masked 버전)
    2. S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010_NBR.tif (원본 버전)
    3. 20151028_R103_T52SCD_NBR.tif (prefix 없는 버전)
    """
    file_map = {}
    if not os.path.exists(directory):
        print(f"   경고: 경로가 존재하지 않습니다: {directory}")
        return file_map

    for tif_file in glob.glob(os.path.join(directory, "*_NBR.tif")):
        basename = os.path.basename(tif_file)
        
        # 패턴 1: masked_YYYYMMDD_RXXX_TXXXXX_NBR.tif
        if basename.startswith("masked_"):
            match = re.search(r'masked_(\d{8}_R\d+_T\w+)_NBR\.tif', basename)
            if match:
                short_name = match.group(1)
                file_map[short_name] = tif_file
                continue
        
        # 패턴 2: S2X_MSIL2A_YYYYMMDDTHHMMSS_NXXXX_RXXX_TXXXXX_YYYYMMDDTHHMMSS_NBR.tif
        match = re.search(r'S2[AB]_MSIL2A_(\d{8})T\d+_N\d+_(R\d+)_(T\w+)_\d+T\d+_NBR\.tif', basename)
        if match:
            date = match.group(1)
            orbit = match.group(2)
            tile = match.group(3)
            short_name = f"{date}_{orbit}_{tile}"
            file_map[short_name] = tif_file
            continue
        
        # 패턴 3: YYYYMMDD_RXXX_TXXXXX_NBR.tif (prefix 없음)
        match = re.search(r'^(\d{8}_R\d+_T\w+)_NBR\.tif$', basename)
        if match:
            short_name = match.group(1)
            file_map[short_name] = tif_file

    return file_map

pre_file_map = build_file_mapping(pre_masked_NBR_dir)
post_file_map = build_file_mapping(post_masked_NBR_dir)

print(f"   Pre-NBR 파일: {len(pre_file_map)}개")
print(f"   Post-NBR 파일: {len(post_file_map)}개")

# Pair 리스트 생성
pair_list = []
unpaired_cases = []

for idx, row in df_ref.iterrows():
    fire_id = row['fire_id']
    fire_date = row['fire_end_date']
    pre_product = row.get(pre_id_column)
    post_product = row.get(post_id_column)

    pre_short = extract_short_name(pre_product)
    post_short = extract_short_name(post_product)

    # Pair 유효성 체크
    has_pre = pre_short is not None and pre_short in pre_file_map
    has_post = post_short is not None and post_short in post_file_map

    if has_pre and has_post:
        # 유효한 Pair
        pair_list.append({
            'fire_id': fire_id,
            'fire_date': fire_date,
            'pre_short_name': pre_short,
            'post_short_name': post_short,
            'pre_file': pre_file_map[pre_short],
            'post_file': post_file_map[post_short]
        })
    else:
        # Unpaired 케이스
        reason = []
        if pre_short is None:
            reason.append("pre_product_id 없음")
        elif pre_short not in pre_file_map:
            reason.append(f"pre 파일 없음 ({pre_short})")

        if post_short is None:
            reason.append("post_product_id 없음")
        elif post_short not in post_file_map:
            reason.append(f"post 파일 없음 ({post_short})")

        unpaired_cases.append({
            'fire_id': fire_id,
            'fire_date': fire_date,
            'pre_product_id': pre_product,
            'post_product_id': post_product,
            'reason': '; '.join(reason)
        })

print(f"\n   완료: Pair 리스트 생성")
print(f"      유효한 Pair: {len(pair_list)}개")
print(f"      Unpaired: {len(unpaired_cases)}개\n")

# Unpaired 케이스 로그 저장
if unpaired_cases:
    with open(unpaired_log, 'w', encoding='utf-8') as f:
        f.write("="*70 + "\n")
        f.write("Unpaired 산불-영상 케이스 로그\n")
        f.write("="*70 + "\n\n")
        f.write(f"작성 일시: {pd.Timestamp.now()}\n")
        f.write(f"총 Unpaired 케이스: {len(unpaired_cases)}개\n\n")

        for case in unpaired_cases:
            f.write(f"[Fire ID: {case['fire_id']}]\n")
            f.write(f"  산불 진화일: {case['fire_date']}\n")
            f.write(f"  Pre-Product ID: {case['pre_product_id']}\n")
            f.write(f"  Post-Product ID: {case['post_product_id']}\n")
            f.write(f"  사유: {case['reason']}\n\n")

    print(f"   Unpaired 로그 저장: {unpaired_log}\n")

if len(pair_list) == 0:
    print("오류: 처리할 유효한 Pair가 없습니다.")
    raise SystemExit("No valid pairs to process")

# ============================================================
# 2단계: dNBR 계산 함수 정의
# ============================================================
print("[2/4] dNBR 계산 함수 정의...")

def calculate_dnbr_intersection(pre_file, post_file, fire_id, pre_name, post_name, output_dir):
    """
    두 NBR 영상의 겹치는 영역(intersection)에 대해 dNBR 계산

    Parameters:
        pre_file: Pre-fire NBR 파일 경로
        post_file: Post-fire NBR 파일 경로
        fire_id: 산불 ID
        pre_name: Pre 짧은 이름
        post_name: Post 짧은 이름
        output_dir: 출력 디렉토리

    Returns:
        success: 성공 여부 (bool)
        message: 결과 메시지
    """
    try:
        # Pre/Post 파일 열기
        with rasterio.open(pre_file) as pre_src, rasterio.open(post_file) as post_src:

            # 두 영상의 경계(bounds) 가져오기
            pre_bounds = pre_src.bounds
            post_bounds = post_src.bounds

            # Intersection (겹치는 영역) 계산
            inter_left = max(pre_bounds.left, post_bounds.left)
            inter_bottom = max(pre_bounds.bottom, post_bounds.bottom)
            inter_right = min(pre_bounds.right, post_bounds.right)
            inter_top = min(pre_bounds.top, post_bounds.top)

            # 겹치는 영역이 있는지 확인
            if inter_left >= inter_right or inter_bottom >= inter_top:
                return False, "No intersection between pre and post images"

            inter_bounds = (inter_left, inter_bottom, inter_right, inter_top)

            # CRS가 다른 경우 처리
            if pre_src.crs != post_src.crs:
                # Post를 Pre의 CRS로 변환
                post_crs = pre_src.crs
            else:
                post_crs = pre_src.crs

            # Pre 영상: intersection 영역만 읽기 (window 사용)
            pre_window = pre_src.window(*inter_bounds)
            pre_data = pre_src.read(1, window=pre_window).astype(np.float32)
            pre_transform = pre_src.window_transform(pre_window)

            # Post 영상: intersection 영역만 읽기 및 리프로젝션
            post_window = post_src.window(*inter_bounds)
            post_data_raw = post_src.read(1, window=post_window).astype(np.float32)
            post_transform_raw = post_src.window_transform(post_window)

            # Post를 Pre 해상도/그리드로 맞추기 (reproject)
            post_data = np.empty_like(pre_data)
            reproject(
                source=post_data_raw,
                destination=post_data,
                src_transform=post_transform_raw,
                src_crs=post_src.crs,
                dst_transform=pre_transform,
                dst_crs=post_crs,
                resampling=Resampling.bilinear,
                src_nodata=np.nan,
                dst_nodata=np.nan
            )

            # dNBR 계산: PRE - POST (산불로 인해 NBR이 감소하므로)
            dnbr = pre_data - post_data

            # 유효 픽셀 마스크 (Pre와 Post 모두 유효한 픽셀만)
            valid_mask = ~np.isnan(pre_data) & ~np.isnan(post_data)
            dnbr[~valid_mask] = np.nan

            # 통계 계산
            valid_pixels = np.sum(valid_mask)
            if valid_pixels == 0:
                return False, "No valid pixels in intersection"

            dnbr_min = np.nanmin(dnbr)
            dnbr_max = np.nanmax(dnbr)
            dnbr_mean = np.nanmean(dnbr)
            dnbr_std = np.nanstd(dnbr)

            # 출력 파일명 생성
            output_filename = f"dNBR_fire{fire_id:04d}_{pre_name}_to_{post_name}.tif"
            output_path = os.path.join(output_dir, output_filename)

            # 메타데이터 설정
            profile = pre_src.profile.copy()
            profile.update({
                'driver': 'GTiff',
                'dtype': rasterio.float32,
                'count': 1,
                'width': pre_data.shape[1],
                'height': pre_data.shape[0],
                'crs': post_crs,
                'transform': pre_transform,
                'compress': 'lzw',
                'nodata': np.nan
            })

            # 저장
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(dnbr, 1)
                dst.set_band_description(1, 'dNBR (Pre - Post)')
                dst.update_tags(1,
                                FIRE_ID=str(fire_id),
                                PRE_IMAGE=pre_name,
                                POST_IMAGE=post_name,
                                FORMULA='PRE_NBR - POST_NBR',
                                VALID_PIXELS=str(valid_pixels),
                                DNBR_MIN=f'{dnbr_min:.6f}',
                                DNBR_MAX=f'{dnbr_max:.6f}',
                                DNBR_MEAN=f'{dnbr_mean:.6f}',
                                DNBR_STD=f'{dnbr_std:.6f}')

            result_msg = (f"OK: dNBR: [{dnbr_min:.3f}, {dnbr_max:.3f}], "
                        f"Mean: {dnbr_mean:.3f}, Valid: {valid_pixels:,}px")

            return True, result_msg

    except Exception as e:
        error_msg = f"Error: {str(e)[:100]}"
        return False, error_msg

print("   완료: 함수 정의\n")

# ============================================================
# 3단계: dNBR 계산 수행
# ============================================================
print(f"[3/4] dNBR 계산 수행 중 ({len(pair_list)}개 Pair)...\n")

results = []
success_count = 0
fail_count = 0

for pair in tqdm(pair_list, desc="   처리 중"):
    fire_id = pair['fire_id']
    pre_file = pair['pre_file']
    post_file = pair['post_file']
    pre_name = pair['pre_short_name']
    post_name = pair['post_short_name']

    success, message = calculate_dnbr_intersection(
        pre_file, post_file, fire_id, pre_name, post_name, output_dir
    )

    if success:
        success_count += 1
    else:
        fail_count += 1

    results.append({
        'fire_id': fire_id,
        'fire_date': pair['fire_date'],
        'pre_name': pre_name,
        'post_name': post_name,
        'success': success,
        'message': message
    })

print(f"\n   완료: 계산 완료 - {success_count}개 성공, {fail_count}개 실패\n")

# ============================================================
# 4단계: 결과 저장
# ============================================================
print("[4/4] 결과 저장 중...")

df_results = pd.DataFrame(results)

# 결과 CSV 저장
results_csv = os.path.join(output_dir, "dnbr_results.csv")
df_results.to_csv(results_csv, index=False, encoding='utf-8-sig')
print(f"   완료: 결과 CSV 저장 - {results_csv}")

# 최종 리포트 저장
report_path = os.path.join(output_dir, "dnbr_report.txt")
with open(report_path, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("dNBR 계산 리포트\n")
    f.write("="*70 + "\n\n")
    f.write(f"작업 일시: {pd.Timestamp.now()}\n")
    f.write(f"CSV 파일: {reference_csv}\n")
    f.write(f"출력 경로: {output_dir}\n\n")

    f.write(f"[처리 결과]\n")
    f.write(f"  총 레코드 수: {len(df_ref)}개\n")
    f.write(f"  유효한 Pair: {len(pair_list)}개\n")
    f.write(f"  Unpaired: {len(unpaired_cases)}개\n")
    f.write(f"  dNBR 계산 성공: {success_count}개\n")
    f.write(f"  dNBR 계산 실패: {fail_count}개\n\n")

    if fail_count > 0:
        f.write(f"[실패한 케이스]\n")
        failed = df_results[df_results['success'] == False]
        for _, row in failed.iterrows():
            f.write(f"  Fire ID {row['fire_id']}: {row['message']}\n")

print(f"   완료: 리포트 저장 - {report_path}")

print("\n" + "="*70)
print("=== dNBR 계산 완료 ===")
print("="*70)
print(f"\n 총 {success_count}개 dNBR 파일이 생성되었습니다.")
print(f" 출력 경로: {output_dir}")
print(f"\n 주요 파일:")
print(f"   - dNBR 결과: {results_csv}")
print(f"   - 처리 리포트: {report_path}")
if unpaired_cases:
    print(f"   - Unpaired 로그: {unpaired_log}")
print()


In [8]:
# RdNBR 계산 셀
import rasterio
import numpy as np
import pandas as pd
import glob
import os
import re
from rasterio.warp import reproject, Resampling
from tqdm import tqdm

reference_csv = r"D:\Landslide\data\raw\산불발생이력\s2_matches-15.csv"
pre_id_column = 'pre_product_id'
post_id_column = 'post_product_id'

pre_masked_NBR_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\pre\NBR"
post_masked_NBR_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR"

output_dir = r"D:\Landslide\data\processed\gyeongnam\S2_outputs\RdNBR"
unpaired_log = os.path.join(output_dir, "unpaired.log")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("="*70)
print("=== RdNBR 계산 시작 ===")
print("="*70)
print(f"CSV 파일: {reference_csv}")
print(f"Pre-NBR 경로: {pre_masked_NBR_dir}")
print(f"Post-NBR 경로: {post_masked_NBR_dir}")
print(f"출력 경로: {output_dir}\n")

# ============================================================
# 1단계: CSV 파일 로드 및 Pair 리스트 생성
# ============================================================
print("[1/4] CSV 파일 로드 및 Pair 리스트 생성 중...")

df_ref = pd.read_csv(reference_csv, encoding='utf-8-sig')
print(f"   총 레코드 수: {len(df_ref)}개")

# 짧은 파일명 추출 함수 (20151028_R103_T52SCD 형식으로 변환)
def extract_short_name(product_id):
    """
    S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010
    -> 20151028_R103_T52SCD
    """
    if pd.isna(product_id) or product_id == '':
        return None

    match = re.search(r'(\d{8})T\d+_N\d+_(R\d+)_(T\w+)_', str(product_id))
    if match:
        date = match.group(1)
        orbit = match.group(2)
        tile = match.group(3)
        return f"{date}_{orbit}_{tile}"
    return None

# 파일명 디렉토리 매핑 생성
def build_file_mapping(directory):
    """디렉토리 내 파일명 -> 전체 경로 매핑
    
    다음 패턴들을 모두 지원:
    1. masked_20151028_R103_T52SCD_NBR.tif (masked 버전)
    2. S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010_NBR.tif (원본 버전)
    3. 20151028_R103_T52SCD_NBR.tif (prefix 없는 버전)
    """
    file_map = {}
    if not os.path.exists(directory):
        print(f"   경고: 경로가 존재하지 않습니다: {directory}")
        return file_map

    for tif_file in glob.glob(os.path.join(directory, "*_NBR.tif")):
        basename = os.path.basename(tif_file)
        
        # 패턴 1: masked_YYYYMMDD_RXXX_TXXXXX_NBR.tif
        if basename.startswith("masked_"):
            match = re.search(r'masked_(\d{8}_R\d+_T\w+)_NBR\.tif', basename)
            if match:
                short_name = match.group(1)
                file_map[short_name] = tif_file
                continue
        
        # 패턴 2: S2X_MSIL2A_YYYYMMDDTHHMMSS_NXXXX_RXXX_TXXXXX_YYYYMMDDTHHMMSS_NBR.tif
        match = re.search(r'S2[AB]_MSIL2A_(\d{8})T\d+_N\d+_(R\d+)_(T\w+)_\d+T\d+_NBR\.tif', basename)
        if match:
            date = match.group(1)
            orbit = match.group(2)
            tile = match.group(3)
            short_name = f"{date}_{orbit}_{tile}"
            file_map[short_name] = tif_file
            continue
        
        # 패턴 3: YYYYMMDD_RXXX_TXXXXX_NBR.tif (prefix 없음)
        match = re.search(r'^(\d{8}_R\d+_T\w+)_NBR\.tif$', basename)
        if match:
            short_name = match.group(1)
            file_map[short_name] = tif_file

    return file_map

pre_file_map = build_file_mapping(pre_masked_NBR_dir)
post_file_map = build_file_mapping(post_masked_NBR_dir)

print(f"   Pre-NBR 파일: {len(pre_file_map)}개")
print(f"   Post-NBR 파일: {len(post_file_map)}개")

# Pair 리스트 생성
pair_list = []
unpaired_cases = []

for idx, row in df_ref.iterrows():
    fire_id = row['fire_id']
    fire_date = row['fire_end_date']
    pre_product = row.get(pre_id_column)
    post_product = row.get(post_id_column)

    pre_short = extract_short_name(pre_product)
    post_short = extract_short_name(post_product)

    # Pair 유효성 체크
    has_pre = pre_short is not None and pre_short in pre_file_map
    has_post = post_short is not None and post_short in post_file_map

    if has_pre and has_post:
        # 유효한 Pair
        pair_list.append({
            'fire_id': fire_id,
            'fire_date': fire_date,
            'pre_short_name': pre_short,
            'post_short_name': post_short,
            'pre_file': pre_file_map[pre_short],
            'post_file': post_file_map[post_short]
        })
    else:
        # Unpaired 케이스
        reason = []
        if pre_short is None:
            reason.append("pre_product_id 없음")
        elif pre_short not in pre_file_map:
            reason.append(f"pre 파일 없음 ({pre_short})")

        if post_short is None:
            reason.append("post_product_id 없음")
        elif post_short not in post_file_map:
            reason.append(f"post 파일 없음 ({post_short})")

        unpaired_cases.append({
            'fire_id': fire_id,
            'fire_date': fire_date,
            'pre_product_id': pre_product,
            'post_product_id': post_product,
            'reason': '; '.join(reason)
        })

print(f"\n   완료: Pair 리스트 생성")
print(f"      유효한 Pair: {len(pair_list)}개")
print(f"      Unpaired: {len(unpaired_cases)}개\n")

# Unpaired 케이스 로그 저장
if unpaired_cases:
    with open(unpaired_log, 'w', encoding='utf-8') as f:
        f.write("="*70 + "\n")
        f.write("Unpaired 산불-영상 케이스 로그\n")
        f.write("="*70 + "\n\n")
        f.write(f"작성 일시: {pd.Timestamp.now()}\n")
        f.write(f"총 Unpaired 케이스: {len(unpaired_cases)}개\n\n")

        for case in unpaired_cases:
            f.write(f"[Fire ID: {case['fire_id']}]\n")
            f.write(f"  산불 진화일: {case['fire_date']}\n")
            f.write(f"  Pre-Product ID: {case['pre_product_id']}\n")
            f.write(f"  Post-Product ID: {case['post_product_id']}\n")
            f.write(f"  사유: {case['reason']}\n\n")

    print(f"   Unpaired 로그 저장: {unpaired_log}\n")

if len(pair_list) == 0:
    print("오류: 처리할 유효한 Pair가 없습니다.")
    raise SystemExit("No valid pairs to process")

# ============================================================
# 2단계: RdNBR 계산 함수 정의
# ============================================================
print("[2/4] RdNBR 계산 함수 정의...")

def calculate_rdnbr_intersection(pre_file, post_file, fire_id, pre_name, post_name, output_dir):
    """
    두 NBR 영상의 겹치는 영역(intersection)에 대해 RdNBR 계산
    
    RdNBR (Relativized dNBR) = dNBR / sqrt(|preNBR|)
    - 산불 전 식생 상태를 고려한 상대적 피해도 지표
    - preNBR 값이 낮을수록 (식생이 적을수록) 더 큰 가중치 적용

    Parameters:
        pre_file: Pre-fire NBR 파일 경로
        post_file: Post-fire NBR 파일 경로
        fire_id: 산불 ID
        pre_name: Pre 짧은 이름
        post_name: Post 짧은 이름
        output_dir: 출력 디렉토리

    Returns:
        success: 성공 여부 (bool)
        message: 결과 메시지
    """
    try:
        # Pre/Post 파일 열기
        with rasterio.open(pre_file) as pre_src, rasterio.open(post_file) as post_src:

            # 두 영상의 경계(bounds) 가져오기
            pre_bounds = pre_src.bounds
            post_bounds = post_src.bounds

            # Intersection (겹치는 영역) 계산
            inter_left = max(pre_bounds.left, post_bounds.left)
            inter_bottom = max(pre_bounds.bottom, post_bounds.bottom)
            inter_right = min(pre_bounds.right, post_bounds.right)
            inter_top = min(pre_bounds.top, post_bounds.top)

            # 겹치는 영역이 있는지 확인
            if inter_left >= inter_right or inter_bottom >= inter_top:
                return False, "No intersection between pre and post images"

            inter_bounds = (inter_left, inter_bottom, inter_right, inter_top)

            # CRS가 다른 경우 처리
            if pre_src.crs != post_src.crs:
                # Post를 Pre의 CRS로 변환
                post_crs = pre_src.crs
            else:
                post_crs = pre_src.crs

            # Pre 영상: intersection 영역만 읽기 (window 사용)
            pre_window = pre_src.window(*inter_bounds)
            pre_data = pre_src.read(1, window=pre_window).astype(np.float32)
            pre_transform = pre_src.window_transform(pre_window)

            # Post 영상: intersection 영역만 읽기 및 리프로젝션
            post_window = post_src.window(*inter_bounds)
            post_data_raw = post_src.read(1, window=post_window).astype(np.float32)
            post_transform_raw = post_src.window_transform(post_window)

            # Post를 Pre 해상도/그리드로 맞추기 (reproject)
            post_data = np.empty_like(pre_data)
            reproject(
                source=post_data_raw,
                destination=post_data,
                src_transform=post_transform_raw,
                src_crs=post_src.crs,
                dst_transform=pre_transform,
                dst_crs=post_crs,
                resampling=Resampling.bilinear,
                src_nodata=np.nan,
                dst_nodata=np.nan
            )

            # dNBR 계산: PRE - POST
            dnbr = pre_data - post_data

            # RdNBR 계산: dNBR / sqrt(|preNBR|)
            # epsilon을 사용하여 0으로 나누는 것 방지
            epsilon = 1e-6
            pre_nbr_abs = np.abs(pre_data)
            
            # preNBR이 매우 작은 경우 epsilon으로 대체
            pre_nbr_abs_safe = np.where(pre_nbr_abs < epsilon, epsilon, pre_nbr_abs)
            
            # RdNBR 계산
            rdnbr = dnbr / np.sqrt(pre_nbr_abs_safe)

            # 유효 픽셀 마스크 (Pre와 Post 모두 유효한 픽셀만)
            valid_mask = ~np.isnan(pre_data) & ~np.isnan(post_data) & np.isfinite(rdnbr)
            rdnbr[~valid_mask] = np.nan

            # 통계 계산
            valid_pixels = np.sum(valid_mask)
            if valid_pixels == 0:
                return False, "No valid pixels in intersection"

            rdnbr_min = np.nanmin(rdnbr)
            rdnbr_max = np.nanmax(rdnbr)
            rdnbr_mean = np.nanmean(rdnbr)
            rdnbr_std = np.nanstd(rdnbr)

            # 출력 파일명 생성
            output_filename = f"RdNBR_fire{fire_id:04d}_{pre_name}_to_{post_name}.tif"
            output_path = os.path.join(output_dir, output_filename)

            # 메타데이터 설정
            profile = pre_src.profile.copy()
            profile.update({
                'driver': 'GTiff',
                'dtype': rasterio.float32,
                'count': 1,
                'width': pre_data.shape[1],
                'height': pre_data.shape[0],
                'crs': post_crs,
                'transform': pre_transform,
                'compress': 'lzw',
                'nodata': np.nan
            })

            # 저장
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(rdnbr, 1)
                dst.set_band_description(1, 'RdNBR (Relativized dNBR)')
                dst.update_tags(1,
                                FIRE_ID=str(fire_id),
                                PRE_IMAGE=pre_name,
                                POST_IMAGE=post_name,
                                FORMULA='dNBR / sqrt(|preNBR|)',
                                VALID_PIXELS=str(valid_pixels),
                                RDNBR_MIN=f'{rdnbr_min:.6f}',
                                RDNBR_MAX=f'{rdnbr_max:.6f}',
                                RDNBR_MEAN=f'{rdnbr_mean:.6f}',
                                RDNBR_STD=f'{rdnbr_std:.6f}')

            result_msg = (f"OK: RdNBR: [{rdnbr_min:.3f}, {rdnbr_max:.3f}], "
                        f"Mean: {rdnbr_mean:.3f}, Valid: {valid_pixels:,}px")

            return True, result_msg

    except Exception as e:
        error_msg = f"Error: {str(e)[:100]}"
        return False, error_msg

print("   완료: 함수 정의\n")

# ============================================================
# 3단계: RdNBR 계산 수행
# ============================================================
print(f"[3/4] RdNBR 계산 수행 중 ({len(pair_list)}개 Pair)...\n")

results = []
success_count = 0
fail_count = 0

for pair in tqdm(pair_list, desc="   처리 중"):
    fire_id = pair['fire_id']
    pre_file = pair['pre_file']
    post_file = pair['post_file']
    pre_name = pair['pre_short_name']
    post_name = pair['post_short_name']

    success, message = calculate_rdnbr_intersection(
        pre_file, post_file, fire_id, pre_name, post_name, output_dir
    )

    if success:
        success_count += 1
    else:
        fail_count += 1

    results.append({
        'fire_id': fire_id,
        'fire_date': pair['fire_date'],
        'pre_name': pre_name,
        'post_name': post_name,
        'success': success,
        'message': message
    })

print(f"\n   완료: 계산 완료 - {success_count}개 성공, {fail_count}개 실패\n")

# ============================================================
# 4단계: 결과 저장
# ============================================================
print("[4/4] 결과 저장 중...")

df_results = pd.DataFrame(results)

# 결과 CSV 저장
results_csv = os.path.join(output_dir, "rdnbr_results.csv")
df_results.to_csv(results_csv, index=False, encoding='utf-8-sig')
print(f"   완료: 결과 CSV 저장 - {results_csv}")

# 최종 리포트 저장
report_path = os.path.join(output_dir, "rdnbr_report.txt")
with open(report_path, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("RdNBR 계산 리포트\n")
    f.write("="*70 + "\n\n")
    f.write(f"작업 일시: {pd.Timestamp.now()}\n")
    f.write(f"CSV 파일: {reference_csv}\n")
    f.write(f"출력 경로: {output_dir}\n\n")

    f.write(f"[처리 결과]\n")
    f.write(f"  총 레코드 수: {len(df_ref)}개\n")
    f.write(f"  유효한 Pair: {len(pair_list)}개\n")
    f.write(f"  Unpaired: {len(unpaired_cases)}개\n")
    f.write(f"  RdNBR 계산 성공: {success_count}개\n")
    f.write(f"  RdNBR 계산 실패: {fail_count}개\n\n")

    if fail_count > 0:
        f.write(f"[실패한 케이스]\n")
        failed = df_results[df_results['success'] == False]
        for _, row in failed.iterrows():
            f.write(f"  Fire ID {row['fire_id']}: {row['message']}\n")

print(f"   완료: 리포트 저장 - {report_path}")

print("\n" + "="*70)
print("=== RdNBR 계산 완료 ===")
print("="*70)
print(f"\n 총 {success_count}개 RdNBR 파일이 생성되었습니다.")
print(f" 출력 경로: {output_dir}")
print(f"\n 주요 파일:")
print(f"   - RdNBR 결과: {results_csv}")
print(f"   - 처리 리포트: {report_path}")
if unpaired_cases:
    print(f"   - Unpaired 로그: {unpaired_log}")
print()


=== RdNBR 계산 시작 ===
CSV 파일: D:\Landslide\data\raw\산불발생이력\s2_matches-15.csv
Pre-NBR 경로: D:\Landslide\data\processed\gyeongnam\S2_outputs\pre\NBR
Post-NBR 경로: D:\Landslide\data\processed\gyeongnam\S2_outputs\post\NBR
출력 경로: D:\Landslide\data\processed\gyeongnam\S2_outputs\RdNBR

[1/4] CSV 파일 로드 및 Pair 리스트 생성 중...
   총 레코드 수: 390개
   Pre-NBR 파일: 128개
   Post-NBR 파일: 150개

   완료: Pair 리스트 생성
      유효한 Pair: 169개
      Unpaired: 221개

   Unpaired 로그 저장: D:\Landslide\data\processed\gyeongnam\S2_outputs\RdNBR\unpaired.log

[2/4] RdNBR 계산 함수 정의...
   완료: 함수 정의

[3/4] RdNBR 계산 수행 중 (169개 Pair)...



   처리 중: 100%|██████████| 169/169 [34:06<00:00, 12.11s/it]


   완료: 계산 완료 - 169개 성공, 0개 실패

[4/4] 결과 저장 중...
   완료: 결과 CSV 저장 - D:\Landslide\data\processed\gyeongnam\S2_outputs\RdNBR\rdnbr_results.csv
   완료: 리포트 저장 - D:\Landslide\data\processed\gyeongnam\S2_outputs\RdNBR\rdnbr_report.txt

=== RdNBR 계산 완료 ===

 총 169개 RdNBR 파일이 생성되었습니다.
 출력 경로: D:\Landslide\data\processed\gyeongnam\S2_outputs\RdNBR

 주요 파일:
   - RdNBR 결과: D:\Landslide\data\processed\gyeongnam\S2_outputs\RdNBR\rdnbr_results.csv
   - 처리 리포트: D:\Landslide\data\processed\gyeongnam\S2_outputs\RdNBR\rdnbr_report.txt
   - Unpaired 로그: D:\Landslide\data\processed\gyeongnam\S2_outputs\RdNBR\unpaired.log

